In [ ]:
import numpy as np
import pandas as pd
import xarray as xr
import rioxarray as rxa
import matplotlib.pyplot as plt
from datetime import datetime


import sys
sys.path.append('/Users/zachkeskinen/Documents/spicy-snow/')
from spicy_snow.IO.user_dates import get_input_dates
from spicy_snow.IO.user_area import get_input_area
from spicy_snow import retrieve_snow_depth

import pickle
with open('/Users/zachkeskinen/Documents/spicy-snow/tests/test_data/2_img_ds', 'rb') as f:
    ds = pickle.load(f)

In [ ]:
x = np.arange(0, 10, 1)
y = np.arange(10, 20, 1)
lon, lat = np.meshgrid(x, y)

ds = xr.Dataset(data_vars= dict(
    data = (["x", "y"], np.random.randn(10, 10))),
           coords = dict(
                x = (["x"], x),
                y = (["y"], y)),
)

In [ ]:
ds.rio.bounds()